# Proyecto integrador I - Raw
---
Presentado por:
* Karla Orozco
* Jonathan zapata
* Juan Fernando Gallego

---
### Importar librerías

In [1]:
import numpy as np
import pandas as pd
import nltk
import re
import matplotlib.pyplot as plt
import seaborn as sns
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('words')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\USUARIO\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\USUARIO\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\USUARIO\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\USUARIO\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

---
### Cargar datos

In [2]:
company = pd.read_csv('raw/Company.csv')
company_tweet = pd.read_csv('raw/Company_Tweet.csv')
tweet = pd.read_csv('raw/Tweet.csv')

---
### Observar datos

In [3]:
company.head()

,ticker_symbol,company_name
0,AAPL,apple
1,GOOG,Google Inc
2,GOOGL,Google Inc
3,AMZN,Amazon.com
4,TSLA,Tesla Inc


In [4]:
company.shape

(6, 2)

In [5]:
company_tweet.head()

,tweet_id,ticker_symbol
0,550803612197457920,AAPL
1,550803610825928706,AAPL
2,550803225113157632,AAPL
3,550802957370159104,AAPL
4,550802855129382912,AAPL


In [6]:
company_tweet.shape

(4336445, 2)

In [7]:
tweet.head()

,tweet_id,writer,post_date,body,comment_num,retweet_num,like_num
0,550441509175443456,VisualStockRSRC,1420070457,"lx21 made $10,008 on $AAPL -Check it out! htt...",0,0,1
1,550441672312512512,KeralaGuy77,1420070496,Insanity of today weirdo massive selling. $aap...,0,0,0
2,550441732014223360,DozenStocks,1420070510,S&P100 #Stocks Performance $HD $LOW $SBUX $TGT...,0,0,0
3,550442977802207232,ShowDreamCar,1420070807,$GM $TSLA: Volkswagen Pushes 2014 Record Recal...,0,0,1
4,550443807834402816,i_Know_First,1420071005,Swing Trading: Up To 8.91% Return In 14 Days h...,0,0,1


In [8]:
tweet.shape

(3717964, 7)

### Reducir el tamaño de la muestra para trabajar más facil:

In [9]:
#tweet=tweet.head(10000).copy()

In [10]:
#company_tweet=company_tweet.loc[company_tweet['tweet_id'].isin(tweet['tweet_id'])]

In [11]:
tweet.shape, company_tweet.shape 

((3717964, 7), (4336445, 2))

Ajustar formato fecha en la tabla "tweet":

In [12]:
tweet['post_date'] = pd.to_datetime(tweet['post_date'],unit='s')
tweet.head(2)

,tweet_id,writer,post_date,body,comment_num,retweet_num,like_num
0,550441509175443456,VisualStockRSRC,2015-01-01 00:00:57,"lx21 made $10,008 on $AAPL -Check it out! htt...",0,0,1
1,550441672312512512,KeralaGuy77,2015-01-01 00:01:36,Insanity of today weirdo massive selling. $aap...,0,0,0


Crear una columna con la fecha hasta la unidad de días:

In [13]:
tweet['date'] = tweet['post_date'].dt.date
tweet.head(2)

,tweet_id,writer,post_date,body,comment_num,retweet_num,like_num,date
0,550441509175443456,VisualStockRSRC,2015-01-01 00:00:57,"lx21 made $10,008 on $AAPL -Check it out! htt...",0,0,1,2015-01-01
1,550441672312512512,KeralaGuy77,2015-01-01 00:01:36,Insanity of today weirdo massive selling. $aap...,0,0,0,2015-01-01


Valores nulos en la tabla "tweet":

In [14]:
tweet.isnull().sum()

tweet_id           0
writer         47273
post_date          0
body               0
comment_num        0
retweet_num        0
like_num           0
date               0
dtype: int64

Evaluar duplicados de la columna "tweet_id" de la tabla "tweet":

In [15]:
tweet['tweet_id'].duplicated().sum()

0

---
### Traer en la tabla "tweet" las compañias de las que habla cada tweet por la columna "tweet_id":

Valores nulos en la tabla "company_tweet":

In [16]:
company_tweet.isnull().sum()

tweet_id         0
ticker_symbol    0
dtype: int64

Evaluar duplicados de la columna "tweet_id" de la tabla "company_tweet":

In [17]:
company_tweet.duplicated(subset=['tweet_id']).sum()

618481

##### Examinar duplicados:

In [18]:
company_tweet.loc[company_tweet.duplicated(subset=['tweet_id'])].head(1)

,tweet_id,ticker_symbol
1425013,550447850857828352,GOOG


In [19]:
company_tweet.loc[company_tweet['tweet_id'] == 550447850857828352]

,tweet_id,ticker_symbol
283,550447850857828352,AAPL
1425013,550447850857828352,GOOG
2145152,550447850857828352,AMZN
3239585,550447850857828352,TSLA


In [20]:
tweet.loc[tweet['tweet_id'] == 550447850857828352, 'body'].values

array(['Top 10 searched #stocks of #2014 $AAPL $FB $BABA $TSLA $BAC $GOOG $INTC $GE $TWTR $AMZN http://stks.co/g1V5S http://stks.co/h1VAb'],
      dtype=object)

Los tweets pueden hacer referencia a más de una empresa.

#### Agrupar por "tweet_id" el "ticker_symbol" de las empresas relacionadas:

In [21]:
ticker_symbol_group = company_tweet.groupby('tweet_id')['ticker_symbol'].unique()

Renombrar y mostrar la serie:

In [22]:
ticker_symbol_group.rename("ticker_symbol_group", inplace=True)
ticker_symbol_group.head(2)

tweet_id
550441509175443456    [AAPL]
550441672312512512    [AAPL]
Name: ticker_symbol_group, dtype: object

Realizar combinación mencionada entre la tabla agrupada "ticker_symbol_group" y "tweet" por la columna "tweet_id":

In [23]:
tweet = tweet.merge(ticker_symbol_group, how='left',left_on='tweet_id',right_on='tweet_id')

Revisar el tweet examinado en los duplicados de la tabla "company_tweet":

In [24]:
tweet.loc[tweet['tweet_id'] == 550447850857828352]

,tweet_id,writer,post_date,body,comment_num,retweet_num,like_num,date,ticker_symbol_group
17,550447850857828352,JorelLaraKalel,2015-01-01 00:26:09,Top 10 searched #stocks of #2014 $AAPL $FB $BA...,0,0,2,2015-01-01,"[AAPL, GOOG, AMZN, TSLA]"


#### Tokenización

Remover los tiker symbol o las URL del cuerpo de los tweets:

In [25]:
tweet['body_trusted']=tweet['body'].apply(lambda x: re.sub(r'\$\S+|http\S+','',x)) #Remover los ticker symbol o las URL

Tokenizar:

In [26]:
tweet['body_tokenized'] = tweet['body_trusted'].apply(lambda x: nltk.word_tokenize(str(x)) )

In [27]:
# stopwords en nltk
stop_words_nltk = set(stopwords.words('english'))
print(f'Stopwords length: {len(stop_words_nltk)}')
print(f'Stopwords: {stop_words_nltk}')

Stopwords length: 179
Stopwords: {'its', 'ma', 'them', "needn't", 'if', 'other', 'hers', "you're", 'an', 'than', 'itself', 'there', 'is', 'when', 'where', "wouldn't", 'she', 'any', 'me', 'weren', 'has', 'having', "you'd", "she's", 'own', 'you', 'myself', 'as', 'needn', "doesn't", 'some', 've', 'off', 'down', 'no', 'and', 'just', 'being', 'we', 'more', 'with', 'between', 'the', 'am', 'themselves', 'into', 'on', 'such', 'further', 'during', 'should', 'aren', "shouldn't", 'in', 'shouldn', 'wouldn', 'hasn', 'o', 'then', 'ours', 'doing', 'herself', 'at', 'a', 'same', 'himself', 'too', 're', 'again', "weren't", "didn't", 'few', 'y', 'him', "mustn't", 'her', 'do', 'once', 'who', 'haven', "hasn't", 'it', 'not', 'here', 'of', 'mightn', 'only', 'how', 'this', "mightn't", 'are', 'why', 'he', 'm', 'to', 'does', "isn't", 't', 'been', "that'll", 'don', 'which', 'most', 'they', 'under', 'up', 'had', "couldn't", 's', 'have', 'over', 'his', 'can', "wasn't", 'after', 'theirs', 'these', 'won', "hadn't", 

Refinar los tokens:

In [28]:
# ELIMINAR caracteres que no pertenezcal al alfabeto
# TRANFORMAR tokens a minuscula y ELIMINAR tokens de long = 1
# REMOVER stop words
tweet['body_refined']=tweet['body_tokenized'] #Crear columna igual a "body_tokenized"
tweet['body_refined']=tweet['body_refined'].apply(lambda tokens: [w for w in tokens if w.isalpha()] ) #alphabet letters
tweet['body_refined']=tweet['body_refined'].apply(lambda tokens: [w.lower() for w in tokens if len(w)>1] ) #lower and len > 1
tweet['body_refined']=tweet['body_refined'].apply(lambda tokens: [w for w in tokens if w not in stop_words_nltk] ) #not stop word

Examinar algún tweet refinado:

In [29]:
tweet.loc[5].at['body'], tweet.loc[5].at['body_refined']

('Swing Trading: Up To 8.91% Return In 14 Days http://ow.ly/GDkrL #swingtrading #forecast #techstock $MWW $AAPL $TSLA',
 ['swing',
  'trading',
  'return',
  'days',
  'swingtrading',
  'forecast',
  'techstock'])

In [30]:
tweet.head(2)

,tweet_id,writer,post_date,body,comment_num,retweet_num,like_num,date,ticker_symbol_group,body_trusted,body_tokenized,body_refined
0,550441509175443456,VisualStockRSRC,2015-01-01 00:00:57,"lx21 made $10,008 on $AAPL -Check it out! htt...",0,0,1,2015-01-01,[AAPL],lx21 made on -Check it out! Learn #howtotr...,"[lx21, made, on, -Check, it, out, !, Learn, #,...","[made, learn, howtotrade]"
1,550441672312512512,KeralaGuy77,2015-01-01 00:01:36,Insanity of today weirdo massive selling. $aap...,0,0,0,2015-01-01,[AAPL],Insanity of today weirdo massive selling. bid...,"[Insanity, of, today, weirdo, massive, selling...","[insanity, today, weirdo, massive, selling, bi..."


Lemmatizar los tokens:

In [31]:
# Lemmatization con NLTK
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()

In [32]:
tweet['lemmatized_tokens']=tweet['body_refined'].apply(lambda tokens: [wordnet_lemmatizer.lemmatize(w) for w in tokens] )

Concatenar los tokens preparados:

In [33]:
tweet['clean_tweet'] = tweet['lemmatized_tokens'].apply(lambda tokens: ' '.join(tokens))

In [34]:
tweet['clean_tweet'].head()

0                                made learn howtotrade
1    insanity today weirdo massive selling bid cent...
2                                    stock performance
3    volkswagen push record recall tally higher pro...
4    swing trading return day swingtrading forecast...
Name: clean_tweet, dtype: object

#### Análisis de sentimientos:

In [35]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\USUARIO\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [36]:
len(tweet)

3717964

In [37]:
tweet['score_vader'] = tweet['clean_tweet'].apply(lambda c: SentimentIntensityAnalyzer().polarity_scores(c)['compound'])

In [38]:
tweet['score_vader'].head()

0    0.0000
1   -0.8271
2    0.0000
3    0.0000
4    0.0000
Name: score_vader, dtype: float64

In [39]:
conditions = [
    (tweet["score_vader"] >= .05),
    (tweet["score_vader"] > -.05) & (tweet["score_vader"] < .05),
    (tweet["score_vader"] <= -.05),
]

choices = ['positive', 'neutral', 'negative']

tweet['sentiment'] = np.select(conditions, choices)

In [40]:
tweet['sentiment'].value_counts()

positive    1555315
neutral     1528212
negative     634437
Name: sentiment, dtype: int64

#### Observar datos

In [41]:
tweet.head(2)

,tweet_id,writer,post_date,body,comment_num,retweet_num,like_num,date,ticker_symbol_group,body_trusted,body_tokenized,body_refined,lemmatized_tokens,clean_tweet,score_vader,sentiment
0,550441509175443456,VisualStockRSRC,2015-01-01 00:00:57,"lx21 made $10,008 on $AAPL -Check it out! htt...",0,0,1,2015-01-01,[AAPL],lx21 made on -Check it out! Learn #howtotr...,"[lx21, made, on, -Check, it, out, !, Learn, #,...","[made, learn, howtotrade]","[made, learn, howtotrade]",made learn howtotrade,0.0000,neutral
1,550441672312512512,KeralaGuy77,2015-01-01 00:01:36,Insanity of today weirdo massive selling. $aap...,0,0,0,2015-01-01,[AAPL],Insanity of today weirdo massive selling. bid...,"[Insanity, of, today, weirdo, massive, selling...","[insanity, today, weirdo, massive, selling, bi...","[insanity, today, weirdo, massive, selling, bi...",insanity today weirdo massive selling bid cent...,-0.8271,negative


#### Calcular total engagement:

In [45]:
tweet['total_engagement']=tweet['comment_num']+tweet['retweet_num']+tweet['like_num']

---
#### Exportar datos a la zona trusted

In [46]:
tweet.to_csv('trusted/data.csv',index=False)

In [47]:
tweet.shape

(3717964, 17)

#### Webgrafías de interés:
* https://aws.amazon.com/es/premiumsupport/knowledge-center/glue-version2-external-python-libraries/
* https://mungingdata.com/pyspark/column-to-list-collect-tolocaliterator/
* https://stackoverflow.com/questions/54373335/aws-glue-crawler-cannot-extract-csv-headers
* https://jakevdp.github.io/PythonDataScienceHandbook/04.14-visualization-with-seaborn.html
* https://towardsdatascience.com/how-tf-idf-works-3dbf35e568f0
* https://www.datacamp.com/community/tutorials/recommender-systems-python
* https://intellica-ai.medium.com/comparison-of-different-word-embeddings-on-text-similarity-a-use-case-in-nlp-e83e08469c1c